#setup:
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [ ]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [ ]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

#Open some corrected_behavior data and mount_events data:

In [ ]:
video_name = 'PZ71_1'

corrected_behavior_file = open(output_dir + video_name + '_corrected_behavior.pickle', 'rb')
corrected_behavior = pickle.load(corrected_behavior_file)
corrected_behavior_file.close()

mount_events_file = open(output_dir + video_name + '_mount_events.pickle', 'rb')
mount_events = pickle.load(mount_events_file)
mount_events_file.close()

#mount forest features

In [ ]:
# get the index of every mount event
examples_indexes = mount_events.copy()
num_examples = examples_indexes.shape[0]
examples_pandas_indexes = list(examples_indexes.index.values)
examples_end_indexes = [None] * num_examples
for i in examples_pandas_indexes:
  examples_end_indexes[i] = mount_events["events_index"][i+1]
examples_indexes = examples_indexes.reset_index(drop=True)

KeyError: ignored

In [ ]:
video_name = 'PZ80_2'

# area_vec = [area_startx, area_starty, area_distx, area_disty]
female_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_female_side.mat")
female_side_vec = female_side_mat['croprect'][0]
male_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_male_side.mat")
male_side_vec = male_side_mat['croprect'][0]

h5_file = pd.read_hdf(project_path +'/videos/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
[nframes, ncols] = h5_file.shape

In [ ]:
# get all data ready for accessing as needed later on
mouse1_feature_points = {}
mouse1_feature_points['mus1_snout'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_shoulder'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine1'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine2'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine3'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine4'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_tailbase'] = get_data('mus1', 'snout', h5_file)

mouse2_feature_points = {}
mouse2_feature_points['mus2_snout'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_shoulder'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine1'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine2'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine3'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine4'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_tailbase'] = get_data('mus2', 'snout', h5_file)

mouse3_feature_points = {}
mouse3_feature_points['mus3_snout'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['mus3_shoulder'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['mus3_spine1'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['mus3_spine2'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['mus3_spine3'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['mus3_spine4'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['mus3_tailbase'] = get_data('mus3', 'snout', h5_file)

- 7 points on the male
- 7 points on the female
- male points to every female point: 7 * 7 = 49 distances
- angles between every 3 points?

In [ ]:
features = pd.DataFrame(index=range(num_examples),columns=range(49+49+2))

In [ ]:
print(examples_indexes[0])
print()

583
0


In [ ]:
# loop may take a while

for i in range(0, num_examples):
  no_mounting_mouse = 0
  no_recipient_mouse = 0
  at_start = examples_indexes[i]
  right_before = examples_indexes[i] - 1
  halfsecond_before = examples_indexes[i] - 15
  onesecond_before = examples_indexes[i] - 30
  right_after = examples_end_indexes[i]
  halfsecond_after = examples_end_indexes[i] + 15

  if within_area(male_side_vec, mouse1_feature_points['mus1_snout'].loc[[at_start]]):
    mounting_mouse = 'mus1'
    mounting_mouse_feature_points = mouse1_feature_points
  elif within_area(male_side_vec, mouse2_feature_points['mus2_snout'].loc[[at_start]]):
    mounting_mouse = 'mus2'
    mounting_mouse_feature_points = mouse2_feature_points
  elif within_area(male_side_vec, mouse3_feature_points['mus3_snout'].loc[[at_start]]):
    mounting_mouse = 'mus3'
    mounting_mouse_feature_points = mouse3_feature_points
  else:
    no_mounting_mouse = 1
    print('no mounting mouse found.')

    
  if (within_area(male_side_vec, mouse1_feature_points['mus1_snout'].loc[[right_before]]) or within_area(male_side_vec, mouse1_feature_points['mus1_shoulder'].loc[[right_before]]) \
  within_area(male_side_vec, mouse1_feature_points['mus1_snout'].loc[[right_after]]) or within_area(male_side_vec, mouse1_feature_points['mus1_shoulder'].loc[[right_after]])) \
  and not mounting_mouse == 'mus1':
    recipient_mouse = 'mus1'
    recipient_mouse_feature_points = mouse1_feature_points
  elif (within_area(male_side_vec, mouse2_feature_points['mus2_snout'].loc[[right_before]]) or within_area(male_side_vec, mouse2_feature_points['mus2_shoulder'].loc[[right_before]]) \
  within_area(male_side_vec, mouse2_feature_points['mus2_snout'].loc[[right_after]]) or within_area(male_side_vec, mouse2_feature_points['mus2_shoulder'].loc[[right_after]])) \
  and not mounting_mouse == 'mus2':
    recipient_mouse = 'mus2'
    recipient_mouse_feature_points = mouse2_feature_points
  elif (within_area(male_side_vec, mouse3_feature_points['mus3_snout'].loc[[right_before]]) or within_area(male_side_vec, mouse3_feature_points['mus3_shoulder'].loc[[right_before]]) \
  within_area(male_side_vec, mouse3_feature_points['mus3_snout'].loc[[right_after]]) or within_area(male_side_vec, mouse3_feature_points['mus3_shoulder'].loc[[right_after]])) \
  and not mounting_mouse == 'mus3':
    recipient_mouse = 'mus3'
    recipient_mouse_feature_points = mouse3_feature_points
  else:
    no_recipient_mouse = 1
    print('no recipient mouse found.')

  col_num = 0
  for mounting_mouse_key, mounting_mouse_point in mounting_mouse_feature_points.items():
      for recipient_moouse_key, recipient_mouse_point in recipient_mouse_feature_points.items():
        row_dist = euclid_dist(mounting_mouse_point.loc[[right_before]], recipient_mouse_point.loc[[right_before]])
        features[col_num][i] = row_dist
        col_num = col_num + 1
        row_dist = euclid_dist(mounting_mouse_point.loc[[right_after]], recipient_mouse_point.loc[[right_before]])
        features[col_num][i] = row_dist
        col_num = col_num + 1
  features[col_num][i] = mounting_mouse_present
  col_num = col_num + 1
  features[col_num][i] = recipient_mouse_present

#sklearn stuff

In [4]:
import numpy as np
training_features = np.array([[7, 2], [3, 4]])
training_labels = np.array([[1, 0]])
print(training_features)
print(training_labels)

[[7 2]
 [3 4]]
[[1 0]]


In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(training_features, training_labels, test_size = 0.25, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

(1000, 20) (1000,)


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Extract the two most important features
important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

##Features:
## 1) Cage side of female mouse at each frame: _on_male_side_
- Location of female mouse's snout
- Overlay of male side in the cage
- Check whether the x and y coordinates of female mouse's snout are within the male side
## 2) Male mounting at each frame: _mount_
- Location of male mouse's snout, shoulders, spine points, and tailbase
- Location of female mouse's spine points and tailbase
- Euclidean distance between all male points and all female point
## 3) Female investigation of the male at each frame: _sniffing_
- Location of female mouse's snout
- Location of male mouse's tailbase and spine points
- Euclidean distance between female mouse's snout and all male mouse points

###Features List:
## _on_male_side_
- female mouse's snout
- male side coordinate boundaries
- __True or False__

## _mount_
- euclid_dist between male snout, shoulder, spine1, spine2, spine3, spine4, tailbase
- & female spine1, spine2, spine3, spine4, tailbase
- __Random Forest?__